# Initialization Codes

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Read the data
X_full = pd.read_csv('../input/recipe-for-rating-predict-food-ratings-using-ml/train.csv', index_col='ID')
X_test = pd.read_csv('../input/recipe-for-rating-predict-food-ratings-using-ml/test.csv', index_col='ID')

y=X_full.Rating
X=X_full.drop("Rating",axis=1)

# Reconfiguring indexes

X1 = X.reset_index(drop=True)
y1 = y.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [4]:
# Null Value Removal
X1.dropna(subset=["Recipe_Review"],inplace=True)
y1=y1.iloc[X1.index]

X1.reset_index(drop=True,inplace=True)
y1.reset_index(drop=True,inplace=True)

# EDA

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
y.value_counts()

In [ ]:
X.isnull().sum()

In [ ]:
X.index.value_counts()

In [ ]:
X.groupby('RecipeNumber')['RecipeCode'].count()

In [ ]:
X.describe()

In [ ]:
X.UserReputation.value_counts()

In [ ]:
X.ReplyCount.value_counts()

In [ ]:
X.BestScore.value_counts()

In [ ]:
X.hist()

In [ ]:
plt.scatter(X.BestScore,y)

* 2 null in Recipe_Review
* X has duplicate IDs(index)... Comment ID performs like an index instead
* Multiple recipe codes present in same recipe number(ranking 1-100)

# Text Analysis and Cleaning

In [5]:
import re

# pattern_presence = pd.DataFrame(index=X.index, columns=patterns.keys())

# for pattern_name, pattern in patterns.items():
#     pattern_presence[pattern_name] = X.str.contains(pattern).astype(int)

# num_dict={col:sum(pattern_presence[col]) for col in pattern_presence.columns}

# pattern_presence
# num_dict

# emoji_counts = X.str.findall(patterns['emoji_pattern']).explode().value_counts()
# out_of_ascii_counts = X.str.findall(patterns['out_of_ascii']).explode().value_counts()
# emoticons = X.str.findall(patterns['emoticons']).explode().value_counts()
# entities = X.str.findall(patterns['html_entities']).explode().value_counts()
# # hanging = X.str.findall(patterns['hanging_parantheses']).explode().value_counts()

# print("Emoji Counts:")
# print(emoji_counts)
# print("\nOut of ASCII Counts:")
# print(out_of_ascii_counts)
# print("\nEmoticons:")
# print(emoticons)
# print("\nEntities:")
# print(entities)
# # print("\nHanging Parantheses:")
# # print(hanging)

# pattern_presence = pd.DataFrame(index=X.index, columns=patterns.keys())

# for pattern_name, pattern in patterns.items():
#     pattern_presence[pattern_name] = X.str.contains(pattern).astype(int)

# num_dict={col:sum(pattern_presence[col]) for col in pattern_presence.columns}

# pattern_presence
# num_dict

# emoji_counts = X.str.findall(patterns['emoji_pattern']).explode().value_counts()
# out_of_ascii_counts = X.str.findall(patterns['out_of_ascii']).explode().value_counts()
# emoticons = X.str.findall(patterns['emoticons']).explode().value_counts()
# entities = X.str.findall(patterns['html_entities']).explode().value_counts()
# # hanging = X.str.findall(patterns['hanging_parantheses']).explode().value_counts()

# print("Emoji Counts:")
# print(emoji_counts)
# print("\nOut of ASCII Counts:")
# print(out_of_ascii_counts)
# print("\nEmoticons:")
# print(emoticons)
# print("\nEntities:")
# print(entities)
# # print("\nHanging Parantheses:")
# # print(hanging)

def replace_patterns(X):

    patterns = {
        'html_entities': r'&[a-zA-Z0-9]+;|&#[a-zA-Z0-9]+;',
        'urls': r'https?://\S+',
        'email_addresses': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
        'html_tags': r'<.*>|</.*>|<.*>(.*?)</.*>',
        'whitespace_pattern': r'\s\s+',
        'emoji_pattern': r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0]+|[\U0001F1E0-\U0001F1FF]{2}|\u200d',
        'emoticons': r'(?::|;|=)(?:-)?(?:\)|\(|D|P)',
        'out_of_ascii': r'[^\x00-\x7F]+'
    }

    replacements = {
        "'": "'",
        """: '"',
        """: '"',
        "é": "e",
        "×": "x",
        "ñ": "n",
        "è": "e",
        "â": "a",
        "½": "0.5",
        "¼": "0.25",
        "⅛": "0.125",
        "'": "'",
        "´": "'",
        r"[:;][o0]\)?": " ",
        r"\.\.*": ", ",
        r"!!*": "!",
        r"--*": "-",
        r",,*": ",",
        r"^re:": "",
        r"\?\?*": "?",
        r"(?<!\\)([_\*~])[\s\S]*?\1": "",
        "&#39;": "'",
        "&amp;": "&",
        "&gt;": " ",
        "&lt;": " ",
        "&#34;": '"'
    }

    contractions = {
        r"\bain't\b": "am not",
        r"\baren't\b": "are not",
        r"\bcan't\b": "can not",
        r"\bcouldn't\b": "could not",
        r"\bdidn't\b": "did not",
        r"\bdoesn't\b": "does not",
        r"\bdon't\b": "do not",
        r"\bhadn't\b": "had not",
        r"\bhasn't\b": "has not",
        r"\bhaven't\b": "have not",
        r"\bhe'd\b": "he would",
        r"\bhe'll\b": "he will",
        r"\bhe's\b": "he is",
        r"\bi'd\b": "i would",
        r"\bi'll\b": "i will",
        r"\bi'm\b": "i am",
        r"\bi've\b": "i have",
        r"\bisn't\b": "is not",
        r"\bit's\b": "it is",
        r"\blet's\b": "let us",
        r"\bma'am\b": "madam",
        r"\bmayn't\b": "may not",
        r"\bmight've\b": "might have",
        r"\bmightn't\b": "might not",
        r"\bmust've\b": "must have",
        r"\bmustn't\b": "must not",
        r"\bshan't\b": "shall not",
        r"\bshe'd\b": "she would",
        r"\bshe'll\b": "she will",
        r"\bshe's\b": "she is",
        r"\bshould've\b": "should have",
        r"\bshouldn't\b": "should not",
        r"\bthat's\b": "that is",
        r"\bthere's\b": "there is",
        r"\bthey'd\b": "they would",
        r"\bthey'll\b": "they will",
        r"\bthey're\b": "they are",
        r"\bthey've\b": "they have",
        r"\bwasn't\b": "was not",
        r"\bwe'd\b": "we would",
        r"\bwe'll\b": "we will",
        r"\bwe're\b": "we are",
        r"\bwe've\b": "we have",
        r"\bweren't\b": "were not",
        r"\bwhat'll\b": "what will",
        r"\bwhat're\b": "what are",
        r"\bwhat's\b": "what is",
        r"\bwhat've\b": "what have",
        r"\bwhere's\b": "where is",
        r"\bwho'll\b": "who will",
        r"\bwho's\b": "who is",
        r"\bwon't\b": "will not",
        r"\bwould've\b": "would have",
        r"\bwouldn't\b": "would not",
        r"\byou'd\b": "you would",
        r"\byou'll\b": "you will",
        r"\byou're\b": "you are",
        r"\byou've\b": "you have"
    }

    custom_stop_words = {
        r"\badded\b": "add",
        r"\bcups\b": "cup",
        r"\bloved\b": "love",
        r"\bloves\b": "love",
        r"\bliked\b": "like",
        r"\brecipes\b": "recipe",
        r"\bthanks\b": "thank",
        r"\btasty\b": "taste",
        r"\bcooked\b": "cook",
        r"\badding\b": "add",
        r"\bmade\b": "make",
        r"\bmaking\b": "make",
        r"\bmakes\b": "make",
        r"\bthought\b": "think",
        r"\btimes\b": "time",
        r"\btried\b": "try",
        r"\bused\b":"use",
    }

    X = X.str.lower()

    X = X.str.replace(patterns['emoji_pattern'], " ", regex=True)
    X = X.str.replace(patterns['emoticons'], " ", regex=True)
    X = X.str.replace(patterns['out_of_ascii'], " ", regex=True)
    X = X.str.replace(patterns['html_tags'], " ", regex=True)
    X = X.str.replace(patterns['urls'], " ", regex=True)
    X = X.str.replace(patterns['email_addresses'], " ", regex=True)

    my_punct = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '.',
        '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_',
        '`', '{', '|', '}', '~', '»', '«', '“', '”']


    X = X.replace(replacements, regex=True)
    X = X.replace(contractions, regex=True)

    punct_pattern = re.compile("[" + re.escape("".join(my_punct)) + "]")
    X=X.apply(lambda x:re.sub(punct_pattern, "", x))
    X = X.replace(custom_stop_words, regex=True)
    with open('../input/dataset/stopwords.txt', 'r') as file:
        stopwords = file.read().splitlines()


    stopwords_pattern = re.compile(r'\b(' + r'|'.join(stopwords) + r')\b\s*')
    X = X.str.replace(stopwords_pattern, '', regex=True)

    X = X.str.replace(patterns['whitespace_pattern'], " ", regex=True)

    return X

{'html_entities': 5010,
 'urls': 26,
 'email_addresses': 6,
 'html_tags': 81,
 'whitespace_pattern': 6291,
 'emoji_pattern': 15,
 'emoticons': 328,
 'out_of_ascii': 594}

Emoji Counts:
Recipe_Review
😋      3
❤      2
😍      2
‍      2
😋❤🤗    1
🙄🤦🏽    1
🥰🥰🥰    1
🥰      1
🤗      1
😅      1
😝      1
🙄😑     1
🤣      1
🤦      1
Name: count, dtype: int64

Out of ASCII Counts:
Recipe_Review
’          711
           156
“           79
”           76
é           42
°           16
…           12
—            7
ñ            7
             5
½            5
¼            4
‘            4
😋            3
è            3
❤️           2
º            2
😍            2
â            2
😅            1
😝            1
🙄😑           1
–            1
⅛            1
🤣            1
🤦‍♀️         1
🥰            1
🤗            1
•            1
🥰🥰🥰          1
🙄🤦🏽‍♀️       1
😋❤🤗          1
☺️           1
´¯           1
¸¸           1
             1
×            1
´            1
Name: count, dtype: int64

Emoticons:
Recipe_Review
:)     243
:-)     40
;)      29
:(      19
=)       8
;-)      7
Name: count, dtype: int64

Entities:
Recipe_Review
&#39;    7209
&#34;    1342
&amp;     775
&gt;        8
&lt;        7
Name: count, dtype: int64

In [6]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer
import string

# Estimator TfidfTransformer should store all parameters as an attribute during init.
#Please remember all these comments when you do another ml project... every one of these took hours to debug!

class TfidfTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, ngram_range=(1, 1), max_features=100):
        self.max_features=max_features
        self.ngram_range=ngram_range
        self.vectorizer = TfidfVectorizer(stop_words=None, lowercase=False, strip_accents=None, ngram_range=self.ngram_range, max_features=self.max_features)

    def fit(self, X, y=None):
        self.vectorizer.fit(X.squeeze())
        return self

    def transform(self, X, y=None):
        return self.vectorizer.transform(X.squeeze())

    def get_feature_names_out(self, input_features=None):
        print(self.vectorizer.get_feature_names_out())
        return self.vectorizer.get_feature_names_out()

# Use this to check correctness but don't take it too seriously.. solve error one by one and then again check if custom transformer works

# from sklearn.utils.estimator_checks import check_estimator
# check_estimator(TfidfTransformer())

In [7]:
# HAVING REPLACE INSIDE OF PIPELINE AND RUNNING IT EVERYTIME DURING CV IS USELESS AS REPLACE CODE IS FIXED AND IT HAS NOTHING TO LEARN FROM THE INDIVIDUAL CV SPLITS
X2=X1.copy(deep=True)
X2.Recipe_Review=replace_patterns(X1.Recipe_Review)

# Pipelines

<ul>
<li>RecipeName: Name of the recipe the comment was posted on -> <b>Dropped</b> </li>
<li>CommentID: Unique ID of the comment -> <b>Dropped</b> </li>
<li>UserName: Name of the user -> <b>Dropped</b> </li>
</ul>

<ul>
    <li>RecipeNumber: Placement of the recipe on the top 100 recipes list -> <b>Not used</b> </li>
<li>RecipeCode: Unique ID of the recipe used by the site -> <b>Not used</b> </li>
<li>UserID: Unique ID of the user who left the comment -> <b>Not used</b></li>
    <li>CreationTimestamp: Time at which the comment was posted as a Unix timestamp -> <b>Not used</b></li>
    <li>BestScore: Score of the comment, likely used by the site to help determine the order comments appear in -> <b>Not used</b></li>
</ul>

<ul>
<li>UserReputation: Internal score of the site, roughly quantifying the past behavior of the user </li>
<li>ReplyCount: Number of replies to the comment</li>
<li>ThumbsUpCount: Number of up-votes the comment has received</li>
<li>ThumbsDownCount: Number of down-votes the comment has received</li>
    <li>Recipe_Review: Text content of the comment</li>
    </ul>
    
* Rating: The score on a 1 to 5 scale that the user gave to the recipe. A score of 0 means that no score was given (Target Variable)


In [23]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.feature_selection import SequentialFeatureSelector,mutual_info_classif,RFE,SelectKBest
from sklearn.model_selection import cross_val_score,RandomizedSearchCV,GridSearchCV

from scipy.stats import uniform, randint

import pickle

rs=42

In [9]:
UserReputation_preprocess=make_pipeline(StandardScaler())
ReplyCount_preprocess=make_pipeline(StandardScaler())
ThumbsUpCount_preprocess=make_pipeline(StandardScaler())
ThumbsDownCount_preprocess=make_pipeline(StandardScaler())

# Using the TfidfVectorizer only for the purposes of 'Splitting' that is separating sentences into words, 'frequency counting', 'tf*idf' and finally 'normalization'
# Manual lemmatization, lowercase conversion,accent stripping, stop word removal has been done via replace_patterns() as stop word removal and identification of tfidf is pathetic in sklearn!
# Another reason is for optimization! Only the above(1st comment) procedures are dynamic that is "need to be repeated in a cv loop"
# 2nd comment procedures remain common be it any split of a cv loop!

# NEVER APPLY TRANSFORMERS PERTAINING TO THE SAME COLUMN IN TWO DIFFERENT ITERATIONS!!!! ex: Applying below functiontransformer in one row and then custom transformer in the next even though both pertain to the same column 'Recipe_Reviews'
# Go with custom transformer whenever you are adding new columns, cos function_transformer's get_feature_names_out causes problems otherwise

preprocess = make_column_transformer(
    (UserReputation_preprocess, ['UserReputation']),
    (ReplyCount_preprocess, ['ReplyCount']),
    (ThumbsUpCount_preprocess, ['ThumbsUpCount']),
    (ThumbsDownCount_preprocess, ['ThumbsDownCount']),
    # (make_pipeline(FunctionTransformer(replace_patterns, feature_names_out='one-to-one'),TfidfTransformer()), ['Recipe_Review']),
    (TfidfTransformer(),['Recipe_Review']),
    remainder='drop',
    n_jobs=-1,
    verbose_feature_names_out=False,
    sparse_threshold=0,
)

In [ ]:
temp=pd.DataFrame(preprocess.fit_transform(X2),columns=preprocess.get_feature_names_out())
temp

# Models

In [16]:
with open('logreg_clf.pkl', 'wb') as file:
    pickle.dump(lg_clf, file)

In [ ]:
with open('../input/dataset/logreg_clf.pkl', 'rb') as file:
    clf = pickle.load(file)

In [20]:
from google.colab import files
from google.colab import drive

with open('lg_clf.pkl', 'wb') as file:
    pickle.dump(lg_clf, file)

try:
    files.download('lg_clf.pkl')
except:
    drive.mount('/content/drive')

    !cp lg_clf.pkl /content/drive/MyDrive/


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## KNeighborsClassifier

In [ ]:
param_grid = {
    'kneighborsclassifier__n_neighbors': range(5,106,10),
    'kneighborsclassifier__weights': ['uniform'],
    'selectkbest__k': [20,30,40],
}

knc_clf= GridSearchCV(make_pipeline(
    preprocess,
    SelectKBest(mutual_info_classif),
    KNeighborsClassifier(),
    ),
                              param_grid,
                              scoring='accuracy',
                              cv=10,
                              n_jobs=-1,
                              refit=True,
                              verbose=4,
                              error_score=-1,
                              return_train_score=True
)

In [ ]:
knc_clf.fit(X1,y1)
print("Best parameters: ", knc_clf.best_params_)
print("Best score: ", knc_clf.best_score_)
print("Mean test scores: ", knc_clf.cv_results_['mean_test_score'])
print("Results: ",knc_clf.cv_results_)

# weights: distance giving >0.9 on entire train while unifrom gives 0.76 on both
# 'distance' did overfit the data with 0.75 test score... best estimator gave test score of 0.7619

## Logistic

In [30]:
param_grid = [
    # {
    # 'logisticregression__C': [1], #f 0.76455927
    # 'logisticregression__max_iter': [10000], #f
    # 'logisticregression__penalty': ['l2'], #f
    # 'logisticregression__solver': ['newton-cholesky'], #f
    # 'logisticregression__class_weight': [None], #f
    # 'logisticregression__tol': [1e-3], #f
    # 'logisticregression__warm_start': [True], #f
    # },
    # {
    # 'logisticregression__C':[0.1,1,10],
    # 'logisticregression__penalty': ['elasticnet'],
    # 'logisticregression__solver': ['saga'],
    # 'logisticregression__class_weight': [None,'balanced'],
    # 'logisticregression__warm_start': [True,False],
    # 'logisticregression__tol': [1e-4],
    # 'logisticregression__l1_ratio': [0.5], #0.7645593735211135
    # # Best parameters:  {'logisticregression__l1_ratio': 0.5, 'logisticregression__penalty': 'elasticnet', 'logisticregression__solver': 'saga', 'logisticregression__tol': 0.0001}
    # },
    # {
    # 'logisticregression__C':[0.1,1,10],

    # 'logisticregression__penalty': ['l1'], #0.76455929
    # 'logisticregression__solver': ['liblinear'],
    # },
    {
        'rfe__n_features_to_select':[10,20,30],
    }
              ]

LogReg=LogisticRegression(multi_class='auto',
                          random_state=rs,
                          verbose=1,
                          n_jobs=-1,
                          dual=False,
                          fit_intercept=True,

                          max_iter=10000,
                          class_weight=None,
                          solver='newton-cholesky',
                          warm_start=True,
                          l1_ratio=None,
                          penalty='l2',
                          tol=1e-3,
                          C=1,
                          intercept_scaling=1,
                           )

lg_clf = GridSearchCV(
    make_pipeline(
        preprocess,
        RFE(LogisticRegression(),
              verbose=1,

              n_features_to_select=20,
              step=1,
              ),
        # LogReg
        ),

    param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    refit=True,
    verbose=4,
    error_score='raise',
    return_train_score=True,
    )


In [ ]:
lg_clf.best_estimator_.named_steps['logisticregression'].coef_

In [ ]:
lg_clf.fit(X1,y1)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [22]:
lg_clf.fit(X1,y1)
print("Best parameters: ", lg_clf.best_params_)
print("Best score: ", lg_clf.best_score_)
print("Mean test scores: ", lg_clf.cv_results_['mean_test_score'])
print("Mean train scores: ", lg_clf.cv_results_['mean_train_score'])
print("Results: ",lg_clf.cv_results_)

# balanced gives pathetic results
# C = 1 seems to be the way
# newton and lbfgs same results but lbfgs faster
# cholesky seems to perform better

Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 

## SVC

In [ ]:
param_grid = {
    'svc__C': [10], #f
    'svc__kernel': ['linear'], #f
    'svc__gamma': ['scale'], #f
    'svc__tol': [1e-3], #f
    'svc__max_iter': [10000] #f
}
svc=SVC()

svc_clf = GridSearchCV(
    make_pipeline(
        preprocess,
        svc,
        ),

    param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    refit=True,
    verbose=4,
    error_score='raise',
    return_train_score=True,
    )


In [ ]:
svc_clf.fit(X1,y1)
print("Best parameters: ", svc_clf.best_params_)
print("Best score: ", svc_clf.best_score_)
print("Mean test scores: ", svc_clf.cv_results_['mean_test_score'])
print("Mean train scores: ", svc_clf.cv_results_['mean_train_score'])
print("Results: ",svc_clf.cv_results_)

#rbf overfitting... linear can't cross 0.7625 but doens't overfit

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters:  {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'linear', 'svc__max_iter': 10000, 'svc__tol': 0.001}
Best score:  0.762505619600845
Mean test scores:  [0.76191892 0.76184563 0.76250562]
Mean train scores:  [0.76219379 0.76289058 0.76358736]
Results:  {'mean_fit_time': array([ 6.6865099 ,  7.27550526, 10.32608929]), 'std_fit_time': array([0.30229687, 0.44329414, 1.42169643]), 'mean_score_time': array([0.87075114, 0.8878777 , 1.11216073]), 'std_score_time': array([0.03343352, 0.0228726 , 0.03939511]), 'param_svc__C': masked_array(data=[0.1, 1, 10],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'param_svc__gamma': masked_array(data=['scale', 'scale', 'scale'],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'param_svc__kernel': masked_array(data=['linear', 'linear', 'linear'],
             mask=[False, False, False],
   

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


# Submit File Generation

In [ ]:
clf.best_estimator_

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=-1, sparse_threshold=0,
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['UserReputation']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['ReplyCount']),
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['ThumbsUpCount']),
                                                 ('pipeline-4',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['ThumbsDownCount']),
                                                 ('tfidftransformer',
                                                  TfidfTransformer(),
                                                  ['Recipe_Review'])],
                                   verbose_feature_names_out=False)),
                ('selectkbest',
                 SelectKBest(k=40,
                             score_func=<function mutual_info_classif at 0x7e57339c5090>)),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=25))])

In [ ]:
# Preprocess the test data
X1_test = X_test.copy(deep=True)
X1_test['Recipe_Review'] = replace_patterns(X_test['Recipe_Review'])

# When you call clf.predict(X1_test), it will apply only the transform methods of the pipeline components to the preprocessed test data X1_test. It will not call the fit methods again.
#The estimator pipeline has already been trained on the entire X_train by gridsearch
# best_estimator_ basically contains the entire pipeline which you passed to Gridsearchcv fitted on the entire X you passed it

predictions = clf.best_estimator_.predict(X1_test)

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Rating': predictions},index=range(1,4547))
output.to_csv('submission.csv', index_label="ID")

In [ ]:
# with open('text.csv', 'w') as file:
#     X2.Recipe_Review.to_csv(file, index=False, header=False)
# with open('X1.csv', 'w') as file:
    # X1.to_csv(file, index=False)